In [4]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# own Modules 
from models.models import LstmMse
from utils.data_loader import DataPreperatorPrediction, DataSet

In [5]:
hyperparam = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path_new" : '../../data/vega_shrinkwrapper_original/NewBlade/', 
        "path_worn" : '../../data/vega_shrinkwrapper_original/WornBlade/'
    },
    "model" : {
        "input_size" : 7,
        "n_hidden" : 150,
        "sequence_size" : 50,
        "batch_size" : 1,
        "lstm_layer" : 3,
    }
}

# Predict new blade data
## Standarize Validation Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

```python
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02,1.45531178e+04, 1.37766733e+03, 6.50149764e-01]
variance_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]
```

### Load and scale training data

In [6]:
new_blade_loader = DataPreperatorPrediction(path=hyperparam['data']['path_new']+'NewBlade001.csv')
preprocessed_new_blade = new_blade_loader.provide_data()

### Load worn blade dataset and scale them with mean and variance of training data

In [7]:
worn_blade_loader = DataPreperatorPrediction(path=hyperparam['data']['path_worn']+'WornBlade001.csv')
preprocessed_worn_blade = worn_blade_loader.provide_data()

In [8]:
dataset_worn_blade = DataSet(preprocessed_worn_blade, timesteps=50)
dataset_new_blade = DataSet(preprocessed_new_blade, timesteps=50)

data_loader_worn_blade = DataLoader(dataset_worn_blade, batch_size=1, num_workers=1, shuffle=False, drop_last=True)
data_loader_new_blade = DataLoader(dataset_new_blade, batch_size=1, num_workers=1, shuffle=False, drop_last=True)

## Predict new blade

In [9]:
# Initiate and load model
model = LstmMse(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'])

PATH = "../../models/MSE_model/best_model_aws_tanh.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create empty dataframe
columns = ["timestamp", "cut_torque_target", "cut_lag error_target", "cut_position_target", "cut_speed_target", 
           "film_position_target", "film_speed_target", "film_lag_error_target", "cut_torque_predicted", 
           "cut_lag error_predicted", "cut_position_predicted", "cut_speed_predicted", "film_position_predicted", 
           "film_speed_predicted", "film_lag_error_predicted"]
df = pd.DataFrame(columns=columns)

print("Start predicting")    
##### Predict #####
for batch_number, data in enumerate(data_loader_new_blade):
    
    input_data, target_data = data
    hidden = model.init_hidden()

    # Forward propagation
    output = model(input_data, hidden)
    
    # Add values to dataframe 
    output = torch.squeeze(output)
    target_data = torch.squeeze(target_data)
    target_data_np = target_data.data.numpy().tolist()
    predicted_data_np = output.data.numpy().tolist()
    data = [batch_number] + target_data_np + predicted_data_np
    df = df.append(pd.Series(data, index=df.columns ), ignore_index=True)

# Save dataframe as csv file
df.to_csv("../visualisation/files/prediction_new_blade_tanh.csv", sep=";", index=False)

print("Finished")  

Start predicting


/Users/carlovoss/jupyter_notebooks/masterarbeit/venv_pm/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Finished


## Predict worn blade

In [25]:
# Initiate and load model
model = LstmMse(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'])

PATH = "../../models/MSE_model/best_model_aws.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create empty dataframe
columns = ["timestamp", "cut_torque_target", "cut_lag error_target", "cut_position_target", "cut_speed_target", 
           "film_position_target", "film_speed_target", "film_lag_error_target", "cut_torque_predicted", 
           "cut_lag error_predicted", "cut_position_predicted", "cut_speed_predicted", "film_position_predicted", 
           "film_speed_predicted", "film_lag_error_predicted"]
df = pd.DataFrame(columns=columns)

print("Start predicting")    
##### Predict #####
for batch_number, data in enumerate(data_loader_worn_blade):
    
    input_data, target_data = data
    hidden = model.init_hidden()

    # Forward propagation
    output = model(input_data, hidden)
    
    # Add values to dataframe 
    output = torch.squeeze(output)
    target_data = torch.squeeze(target_data)
    target_data_np = target_data.data.numpy().tolist()
    predicted_data_np = output.data.numpy().tolist()
    data = [batch_number] + target_data_np + predicted_data_np
    df = df.append(pd.Series(data, index=df.columns ), ignore_index=True)

# Save dataframe as csv file
df.to_csv("../visualisation/files/prediction_worn_blade.csv", sep=";", index=False)

print("Finished") 

Start predicting
Finished
